In [70]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [71]:
import os

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
!pip install py4j
!export SPARK_HOME=/content/spark-3.0.1-bin-hadoop2.7.tgz
!export PATH=$SPARK_HOME:$PATH
!export PYTHONPATH=$SPARK_HOME/python:$PYTHONPATH

     |████████████████████████████████| 204kB 4.0MB/s 


In [72]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [77]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [82]:
import requests
url = "https://projects.fivethirtyeight.com/polls-page/president_polls.csv"
r = requests.get(url, allow_redirects=True)
open('president_polls.csv', 'wb').write(r.content)


5545626

In [83]:

df = spark.read.csv("president_polls.csv",header=True)
df.createOrReplaceTempView("polls")
query = """
SELECT a.start_date,case when a.answer ="Biden" then "poll_biden" else "poll_trump" end as user,weighted_average/total as pct,0-((50-(weighted_average/total))*2)/100 as mean FROM
(SELECT answer,start_date,sum(sample_size*pct) as weighted_average FROM polls as a group by 1,2 ) a
JOIN
(SELECT answer,start_date,sum(sample_size) as total FROM polls as a group by 1,2) b
ON a.answer = b.answer and a.start_date = b.start_date where a.answer in ('Trump','Biden') and a.start_date like "%20"
order by 2,1"""
sqlDF = spark.sql(query)
sqlDF.show()


+----------+----------+------------------+--------------------+
|start_date|      user|               pct|                mean|
+----------+----------+------------------+--------------------+
|   1/10/20|poll_biden|              46.0|               -0.08|
|   1/11/20|poll_biden|              45.0|                -0.1|
|   1/14/20|poll_biden| 52.47903916768666| 0.04958078335373315|
|   1/15/20|poll_biden|  45.9490245437382|-0.08101950912523606|
|   1/16/20|poll_biden|46.848320325755004|-0.06303359348489991|
|   1/19/20|poll_biden|              50.0|                 0.0|
|    1/2/20|poll_biden|              46.0|               -0.08|
|   1/20/20|poll_biden| 47.55653923541247|-0.04886921529175055|
|   1/21/20|poll_biden|46.125032651284286|-0.07749934697431428|
|   1/22/20|poll_biden|              45.0|                -0.1|
|   1/23/20|poll_biden|              49.0|               -0.02|
|   1/26/20|poll_biden|              50.0|                 0.0|
|   1/27/20|poll_biden|              46.

In [85]:
sqlDF.toPandas().to_csv('pyspark_transformed_polls.csv')
